In [333]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
#import matplotlib.pyplot as plt
from sklearn import metrics
import numpy as np


currency_list = [#'USDCHF10080',
                 #'GBPUSD10080', 
                 #'EURUSD10080',
    #'USDJPY10080',
    #'USDCAD10080', 
    #'AUDUSD10080',
    #'NZDUSD10080',
                  #'GBPCHF10080',
    #'EURCHF10080', 
    'CHFJPY10080',
    #'CADCHF10080', 
    #'AUDCHF10080',
    #'NZDCHF10080',
    'EURGBP10080',
   #'GBPJPY10080',
    #'GBPCAD10080',
    #'GBPAUD10080',
    #'EURJPY10080', 
    #'EURCAD10080', 
    #'EURAUD10080', 
    #'EURNZD10080',
#'CADJPY10080', 
    #'AUDJPY10080',
    #'NZDJPY10080',
    #'AUDCAD10080',
#'NZDCAD10080',
#'AUDNZD10080'
                ]





In [334]:
shortest_row_currency = ''
lowest_number_rows = 1000000000000000000

for p in currency_list:
    
    current_currency_row = pd.read_csv('recent_data/' + p + '.csv', names=headers  ) 
    current_currency_row = len( current_currency_row.index )
    
    if current_currency_row < lowest_number_rows:
        
        lowest_number_rows = current_currency_row
        shortest_row_currency = p
        
        
        
print(shortest_row_currency, lowest_number_rows)




EURGBP10080 1469


In [335]:
# #loops through all currency, identify timeframes not in nzdcad and save them in global_available_timeframes list

headers = ['date_start', 'ignore', 'open', 'high', 'low', 'close', 'volume'  ]

shortest_row_currency = pd.read_csv('recent_data/' + p + '.csv', names=headers  ) 

#this reverses the rows so as to have the recent timeframe at the top
#shortest_row_currency = shortest_row_currency.sort_index(axis=0,ascending=False)



shortest_row_currency_timeframes = shortest_row_currency['date_start']



shortest_row_currency_timeframes = shortest_row_currency_timeframes.values.tolist()


global_unavailable_timeframes = []
for x in currency_list:
    
    
    current_currency_data = pd.read_csv('recent_data/' + x + '.csv', names=headers  )  
    
    current_currency_data_timeframes = current_currency_data['date_start']
   
    current_currency_data_timeframes = current_currency_data_timeframes.values.tolist()
   
    for y in shortest_row_currency_timeframes:
        
            if y not in current_currency_data_timeframes :
                
                global_unavailable_timeframes.append(y)
                

# this get the unique time frames because time frames could have been duplicated from different currencies 
global_unavailable_timeframes = np.unique(global_unavailable_timeframes)   


print(len(global_unavailable_timeframes))   
print(len(shortest_row_currency_timeframes))


14
1469


In [336]:
# removing the globally unavailable timeframes from the shortest row currency

for x in global_unavailable_timeframes:
    shortest_row_currency_timeframes.remove(x)
    
globally_available_time_frame = shortest_row_currency_timeframes
print(len(globally_available_time_frame))

1455


In [337]:
# loop through all currency pairs, get the needed rows from them and assemble all rows to form a combine_data_dataframe

combine_data_dataframe = pd.DataFrame(data={}, index = pd.Series(range(0,len(globally_available_time_frame))), )

for x in currency_list :
    
    

    current_currency_data = pd.read_csv('recent_data/' + x + '.csv', names=headers  ) 
    
    # this sort the rows, so as to have the most recent time frame at the top
    current_currency_data = current_currency_data.sort_index(axis=0,ascending=False)
   
    current_currency_volume = []
    
    current_currency_class = []
    
    q = 0

    for y in globally_available_time_frame:
            
        for index, row in current_currency_data.iterrows():
           

            if y  == row.date_start:            
                        
                #current_currency_volume.append(row.volume)            
                
                # condition when the curreny movement is a buy

                pips_corrector = 100000

                if x.endswith('jpy'):# use this to correct multiplier of jpy pairs
                    pips_corrector = 1000

                if row.open < row.close:

                    trend = 1


                    #classifying the risk level


                    if (row.open - row.low) == 0 : # no risk because open is equal to low

                        risk = 6

                    else : 

                        if (row.open - row.low)> (row.high - row.open):

                            risk = 1

                        if  (row.high - row.open) > (row.open - row.low):

                            risk = 2


                        if (row.high - row.open) /(row.open - row.low) >2 :

                            risk = 3

                        if (row.high - row.open) /(row.open - row.low) >4 :

                            risk = 4


                        if (row.high - row.open) /(row.open - row.low) >8 :

                            risk = 5

                        if (row.high - row.open) /(row.open - row.low) >16 :

                            risk = 6  

                    #classifying reward level

                    if (row.close - row.open)*pips_corrector > 0 :

                        reward = 1

                    if (row.close - row.open)*pips_corrector >200 :

                        reward = 2

                    if (row.close - row.open)*pips_corrector > 400 :

                        reward = 3


                    if (row.close - row.open)*pips_corrector > 800 :

                        reward = 4

                    if (row.close - row.open)*pips_corrector > 1600 :

                        reward = 5


                    if (row.close - row.open)*pips_corrector > 3200 :

                        reward = 6  




                # this is for the condition when the movement is a sell
                else: 

                    trend = -1

                    #classifying the risk level


                    if (row.high - row.open) == 0 : # no risk because open is equal to high

                        risk = 6

                    else : 

                        if (row.high - row.open)> (row.open - row.low):

                            risk = 1

                        if  (row.open - row.low) > (row.high - row.open):

                            risk = 2


                        if (row.open - row.low) /(row.high - row.open) >2 :

                            risk = 3

                        if (row.open - row.low) /(row.high - row.open) >4 :

                            risk = 4

                        if (row.open - row.low) /(row.high - row.open) >8 :

                            risk = 5

                        if (row.open - row.low) /(row.high - row.open) >16 :

                            risk = 6 


                     #classifying reward level

                    if (row.open - row.close)*pips_corrector > 0 :

                        reward = 1

                    if (row.open - row.close)*pips_corrector > 200 :

                        reward = 2

                    if (row.open - row.close)*pips_corrector > 400 :

                        reward = 3        

                    if (row.open - row.close)*pips_corrector > 800 :

                        reward = 4    

                    if (row.open - row.close)*pips_corrector > 1600 :

                        reward = 5 

                    if (row.open - row.close)*pips_corrector > 3200 :

                        reward = 6        
                
                #print(index)
                break 
        current_currency_class.append(trend*risk*reward)
        current_currency_volume.append(row.volume)   
        

            #attach the classification score value to the dataframe and train a model with the data
            #print(index)
    #print(current_currency_class)

            #combine_data_dataframe[x+'_class'] = current_currency_class
    
    
            
                        

    # this function takes a list and returns with a list of 5 list that is the data for the previous row
    
    def add_5_columns(real_column) :
        
        current_currency_column_add_on = [real_column, [], [], [], [], []]
        
        g = 1 
        
        while g < 6:
            
            current_currency_column_add_on[g] = current_currency_column_add_on[g-1].copy()
            current_currency_column_add_on[g].pop(0)
            current_currency_column_add_on[g].append(0)    
            g = g+1
            
        current_currency_column_add_on.pop(0)
        return current_currency_column_add_on
            
            
           

    combine_data_dataframe[ x +'_volume'] = current_currency_volume
    
    #volume_columns = add_5_columns(current_currency_volume)
    
    
    
    
    combine_data_dataframe[ x +'_class'] = current_currency_class
    
    #low_columns = add_5_columns(current_currency_low)
    
   
    
    
    
    #combine_data_dataframe[ x +'_close'] = current_currency_close
    
   




    
    print(x)
    
    




CHFJPY10080
EURGBP10080


In [338]:
#create and add the timeframe column to the combine_data_data_frame to be able to identify each rows

date_series = pd.Series(globally_available_time_frame)

# the code below convert it to panda date time object
date_series = pd.to_datetime(date_series)


#turning date_series to a data frame so as to be able to add it as the first column 
date_series_dataframe =  pd.DataFrame(data={}, index = pd.Series(range(0, len(globally_available_time_frame))), )
date_series_dataframe['date_start'] = date_series


    


#the two dataframes are combined along the y axis to form one dataframe with all information and date
combine_data_dataframe = pd.concat([ date_series_dataframe, combine_data_dataframe, ], axis=1)

combine_data_dataframe

,date_start,CHFJPY10080_volume,CHFJPY10080_class,EURGBP10080_volume,EURGBP10080_class
0,1993-05-02,8224,10,4115,4
1,1993-05-09,9885,-18,4665,6
2,1993-05-16,9495,-12,4165,-20
3,1993-05-23,10335,12,4685,8
4,1993-05-30,7314,-12,4215,9
...,...,...,...,...,...
1450,2021-08-01,252543,-12,207898,-4
1451,2021-08-08,207820,-18,176495,6
1452,2021-08-15,249902,24,195055,20
1453,2021-08-22,254803,36,181066,-2


In [339]:
# copies the data frame so as to isolate the first 10 rows for prediction
combine_top_10_rows_dataframe = combine_data_dataframe.copy(deep=True)


combine_top_10_rows_dataframe = combine_top_10_rows_dataframe.head(25)

#saves the first 10 rows to excel

combine_top_10_rows_dataframe.to_excel("files/combine_top_10_rows_dataframe.xlsx")



combine_data_dataframe = combine_data_dataframe.head( len(globally_available_time_frame) - 5 )

combine_data_dataframe.to_excel("files/combine_data_dataframe.xlsx") 
